# TensorFlow Datasets

TFDS provides a collection of ready-to-use datasets for use with TensorFlow, Jax, and other Machine Learning frameworks.

It handles downloading and preparing the data deterministically and constructing a `tf.data.Dataset` (or `np.array`).

Note: Do not confuse [TFDS](https://www.tensorflow.org/datasets) (this library) with `tf.data` (TensorFlow API to build efficient data pipelines). TFDS is a high level wrapper around `tf.data`. If you're not familiar with this API, we encourage you to read [the official tf.data guide](https://www.tensorflow.org/guide/data) first.


Copyright 2018 The TensorFlow Datasets Authors, Licensed under the Apache License, Version 2.0

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/datasets/overview"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/datasets/overview.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/datasets/overview.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/datasets/overview.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

## Instalación

TFDS existe en dos paquetes:

- `pip install tensorflow-datasets`: la versión estable, que se publica cada unos meses.
- `pip install tfds-nightly`: se publica todos los días y contiene las últimas versiones de los conjuntos de datos.

Este colab usa `tfds-nightly`:


In [ ]:
!pip install -q tfds-nightly tensorflow matplotlib

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Buscar conjuntos de datos disponibles

Todos los generadores de conjuntos de datos son subclases de `tfds.core.DatasetBuilder`. Para obtener la lista de los generadores disponibles, use `tfds.list_builders()` o consulte nuestro [catálogo](https://www.tensorflow.org/datasets/catalog/overview).

In [ ]:
tfds.list_builders()

## Cargar un conjunto de datos

### tfds.load

La forma más fácil de cargar un conjunto de datos es `tfds.load` que hará lo siguiente:

1. Descarga los datos y los guarda como archivos [`tfrecord`](https://www.tensorflow.org/tutorials/load_data/tfrecord).
2. Carga el archivo `tfrecord` y crea el `tf.data.Dataset`.


In [ ]:
ds = tfds.load('mnist', split='train', shuffle_files=True)
assert isinstance(ds, tf.data.Dataset)
print(ds)

Algunos argumentos comunes:

- `split=`: que divide para leer (por ejemplo `'train'`, `['train', 'test']`, `'train[80%:]'`,...). Consulte nuestra [guía de API de división](https://www.tensorflow.org/datasets/splits).
- `shuffle_files=`: controla si se aleatorizan los archivos entre cada época (TFDS almacena conjuntos de datos grandes en varios archivos más pequeños).
- `data_dir=`: ubicación donde se guarda el conjunto de datos (el valor predeterminado es `~/tensorflow_datasets/`)
- `with_info=True`: devuelve `tfds.core.DatasetInfo` que contiene los metadatos del conjunto de datos
- `download=False`: deshabilita la descarga


### tfds.builder

`tfds.load` es un contenedor fino alrededor de `tfds.core.DatasetBuilder`. Se puede obtener el mismo resultado con la API `tfds.core.DatasetBuilder`:

In [ ]:
builder = tfds.builder('mnist')
# 1. Create the tfrecord files (no-op if already exists)
builder.download_and_prepare()
# 2. Load the `tf.data.Dataset`
ds = builder.as_dataset(split='train', shuffle_files=True)
print(ds)

### CLI `tfds build`

Si quiere generar un conjunto de datos específico, puede usar la [línea de comando `tfds`](https://www.tensorflow.org/datasets/cli). Por ejemplo:

```sh
tfds build mnist
```

Consulte [el documento](https://www.tensorflow.org/datasets/cli) para conocer los indicadores disponibles.

## Iterar sobre un conjunto de datos

### Como dict

De forma predeterminada, el objeto `tf.data.Dataset` contiene un `dict` de varios `tf.Tensor`:

In [ ]:
ds = tfds.load('mnist', split='train')
ds = ds.take(1)  # Only take a single example

for example in ds:  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
  print(list(example.keys()))
  image = example["image"]
  label = example["label"]
  print(image.shape, label)

Para conocer la estructura y los nombres de las claves del `dict`, consulte la documentación del conjunto de datos en [nuestro catálogo](https://www.tensorflow.org/datasets/catalog/overview#all_datasets). Por ejemplo: [documentación de mnist](https://www.tensorflow.org/datasets/catalog/mnist).

### Como tupla (`as_supervised=True`)

Al usar `as_supervised=True`, puede obtener una tupla `(features, label)` en lugar de conjuntos de datos supervisados.

In [ ]:
ds = tfds.load('mnist', split='train', as_supervised=True)
ds = ds.take(1)

for image, label in ds:  # example is (image, label)
  print(image.shape, label)

### Como numpy (`tfds.as_numpy`)

Usa `tfds.as_numpy` para convertir:

- `tf.Tensor` -&gt; `np.array`
- `tf.data.Dataset` -&gt; `Iterator[Tree[np.array]]` (`Tree` puede ser `Dict`, `Tuple` de anidado arbitrario)


In [ ]:
ds = tfds.load('mnist', split='train', as_supervised=True)
ds = ds.take(1)

for image, label in tfds.as_numpy(ds):
  print(type(image), type(label), label)

### Como tf.Tensor por lotes (`batch_size=-1`)

Al usar `batch_size=-1`, puede cargar el conjunto de datos completo en un solo lote.

Esto se puede combinar con `as_supervised=True` y `tfds.as_numpy` para obtener los datos como `(np.array, np.array)`:


In [ ]:
image, label = tfds.as_numpy(tfds.load(
    'mnist',
    split='test',
    batch_size=-1,
    as_supervised=True,
))

print(type(image), image.shape)

Tenga cuidado de que su conjunto de datos entre en la memoria y que todos los ejemplos tengan la misma forma.

## Realizar la prueba comparativa de sus conjuntos de datos

Realizar la prueba comparativa de un conjunto de datos es una simple llamada a `tfds.benchmark` en cualquier iterable (por ejemplo, `tf.data.Dataset`, `tfds.as_numpy`,...).


In [ ]:
ds = tfds.load('mnist', split='train')
ds = ds.batch(32).prefetch(1)

tfds.benchmark(ds, batch_size=32)
tfds.benchmark(ds, batch_size=32)  # Second epoch much faster due to auto-caching

- No olvide normalizar los resultados por tamaño de lote con el kwarg `batch_size=`.
- En el resumen, se separa el primer lote de calentamiento para capturar el tiempo de configuración adicional de `tf.data.Dataset` (por ejemplo, inicialización de búferes,...).
- Observe cómo la segunda iteración es mucho más rápida debido al [almacenamiento en caché automático de TFDS](https://www.tensorflow.org/datasets/performances#auto-caching).
- `tfds.benchmark` devuelve un `tfds.core.BenchmarkResult` que se puede inspeccionar para realizar un análisis más profundo.

### Construya una canalización de principio a fin

Para profundizar más, puede consultar:

- Nuestro [ejemplo de Keras de principio a fin](https://www.tensorflow.org/datasets/keras_example) para ver un proceso de entrenamiento completo (con procesamiento por lotes, aleatorización,...).
- Nuestra [guía de rendimiento](https://www.tensorflow.org/datasets/performances) para mejorar la velocidad de sus canalizaciones (consejo: use `tfds.benchmark(ds)` para realizar pruebas comparativas de sus conjuntos de datos).


## Visualización

### tfds.as_dataframe

Se pueden convertir los objetos `tf.data.Dataset` en [`pandas.DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) con `tfds.as_dataframe` para poder visualizarlos en [Colab](https://colab.research.google.com).

- Agregue `tfds.core.DatasetInfo` como segundo argumento de `tfds.as_dataframe` para visualizar imágenes, audio, textos, videos,...
- Use `ds.take(x)` para mostrar solo los primeros ejemplos `x`. `pandas.DataFrame` cargará el conjunto de datos completo en la memoria y puede tardar mucho mostrarlo.

In [ ]:
ds, info = tfds.load('mnist', split='train', with_info=True)

tfds.as_dataframe(ds.take(4), info)

### tfds.show_examples

`tfds.show_examples` devuelve `matplotlib.figure.Figure` (ahora solo se admiten conjuntos de datos de imágenes):

In [ ]:
ds, info = tfds.load('mnist', split='train', with_info=True)

fig = tfds.show_examples(ds, info)

## Acceder a los metadatos del conjunto de datos

Todos los generadores incluyen un objeto `tfds.core.DatasetInfo` que contiene los metadatos del conjunto de datos.

Se puede acceder a través de:

- La API `tfds.load`:


In [ ]:
ds, info = tfds.load('mnist', with_info=True)

- La API `tfds.core.DatasetBuilder`:

In [ ]:
builder = tfds.builder('mnist')
info = builder.info

La información del conjunto de datos contiene información adicional sobre el conjunto de datos (versión, cita, página de inicio, descripción,...).

In [ ]:
print(info)

### Metadatos de las características (nombres de etiquetas, forma de la imagen,...)

Acceda al `tfds.features.FeatureDict`:

In [ ]:
info.features

Cantidad de las clases, nombres de las etiquetas:

In [ ]:
print(info.features["label"].num_classes)
print(info.features["label"].names)
print(info.features["label"].int2str(7))  # Human readable version (8 -> 'cat')
print(info.features["label"].str2int('7'))

Formas, dtypes:

In [ ]:
print(info.features.shape)
print(info.features.dtype)
print(info.features['image'].shape)
print(info.features['image'].dtype)

### Dividir los metadatos (por ejemplo, dividir los nombres, número de ejemplos,...)

Acceda al `tfds.core.SplitDict`:

In [ ]:
print(info.splits)

Divisiones disponibles:

In [ ]:
print(list(info.splits.keys()))

Obtenga información sobre una división:

In [ ]:
print(info.splits['train'].num_examples)
print(info.splits['train'].filenames)
print(info.splits['train'].num_shards)

También funciona con la API subsplit:

In [ ]:
print(info.splits['train[15%:75%]'].num_examples)
print(info.splits['train[15%:75%]'].file_instructions)

## Solución de problemas

### Descarga manual (si no se puede descargar)

Si no se descarga por algún motivo (por ejemplo, sin conexión,...). Siempre puede descargar los datos de forma manual por su cuenta y colocarlos en `manual_dir` (el valor predeterminado es `~/tensorflow_datasets/downloads/manual/`.

Para saber qué URL descargar, consulte:

- Para conjuntos de datos nuevos (implementados como carpeta): [`tensorflow_datasets/`](https://github.com/tensorflow/datasets/tree/master/tensorflow_datasets/) `<type>/<dataset_name>/checksums.tsv`. Por ejemplo: [`tensorflow_datasets/datasets/bool_q/checksums.tsv`](https://github.com/tensorflow/datasets/blob/master/tensorflow_datasets/datasets/bool_q/checksums.tsv).

    Puede encontrar la ubicación de origen del conjunto de datos en [nuestro catálogo](https://www.tensorflow.org/datasets/catalog/overview).

- Para conjuntos de datos antiguos: [`tensorflow_datasets/url_checksums/<dataset_name>.txt`](https://github.com/tensorflow/datasets/tree/master/tensorflow_datasets/url_checksums)

### Arreglar `NonMatchingChecksumError`

TFDS garantiza el determinismo al validar las sumas de verificación de las URL descargadas. Si ocurre un error `NonMatchingChecksumError`, puede indicar lo siguiente:

- Es posible que el sitio web esté inactivo (por ejemplo, `503 status code`). Revise la URL.
- Para las URL de Google Drive, inténtelo de nuevo más tarde, ya que, a veces, Drive rechaza las descargas cuando demasiadas personas acceden a la misma URL. Ver [error](https://github.com/tensorflow/datasets/issues/1482)
- Es posible que se hayan actualizado los archivos originales del conjuntos de datos. En este caso, se debe actualizar el generador de conjuntos de datos TFDS. Abra una nueva edición de Github o solicitud de cambios (PR):
    - Registre las nuevas sumas de comprobación con `tfds build --register_checksums`
    - Finalmente actualice el código de generación del conjunto de datos.
    - Actualize la `VERSION` del conjunto de datos
    - Actualize el conjunto de datos `RELEASE_NOTES`: ¿Qué provocó que cambiaran las sumas de comprobación? ¿Cambiaron algunos ejemplos?
    - Asegúrese de que el conjunto de datos aún se pueda construir.
    - Envíenos una solicitud de cambios (PR)

Nota: También puede inspeccionar el archivo descargado en `~/tensorflow_datasets/download/`.


## Cita

Si está usando `tensorflow-datasets` para un artículo, incluya la siguiente cita, además de las citas específicas de los conjuntos de datos que usó (puede encontrarlas en el [catálogo de conjuntos de datos](https://www.tensorflow.org/datasets/catalog/overview)).

```
@misc{TFDS,
  title = { {TensorFlow Datasets}, A collection of ready-to-use datasets},
  howpublished = {\url{https://www.tensorflow.org/datasets}},
}
```